# <font color=darkslategray>Illumina Data Processing (Post-R)
### <font color=darkslategray>*also known as "Pipeline 4"*
#### <font color=darkgreen>Jessie Berta-Thompson & Gary Olds
#### <font color=green>Designed for DBG fungal DNA barcoding projects
##### <font color=mediumseagreen>Last Edited: 2021/05/07

## <font color=magenta>CHECKPOINT
##### <font color=rebeccapurple>*A local working directory (folder) must be set up with the following instructions exactly in order for code in this notebook to run correctly.*

### <font color=dodgerblue>*Program Prep*
Make sure the appropriate programs & tools are installed & imported.

**See ComputerTracking.doc for details.**

<font color=red>This version works specifically for Gary's computer and requires extra downloads to run linux and mac-related things on a PC. See separate doc for computer setup...

### <font color=dodgerblue>*Complete Directory Naming*
Name all directories needed throughout the workflow (only edit MAINdir, all others are automatic)

In [ ]:
UPdir = "C:\\Users\\cgogo\\Documents\\PIPELINE\\MICHIGAN"
MAINdir = "C:\\Users\\cgogo\\Documents\\PIPELINE\\MICHIGAN\\SplitSamples"
TRIMMEDdir = f"{MAINdir}\\NoPrimers"
PROCESSEDdir = f"{TRIMMEDdir}\\Filter_Trim"

# Parse dada2 inference table for most abundant fasta and fact table

**Goals:**<br>
Simple fasta output of most abundant sequence per specimen from dada2 inference table.<br>
Fact table, one line per specimen, about process and about sequences. <br>
**Dataset:**<br>
Testing on Owl pooled data, run with maxEE 2,2 per-sample bimera handling dada2 pipeline.<br>
But writing to be a re-usable, shareable tool for Gary to use on lots of datasets while running methods comparisons.<br>

**Working off code from here for a start:**<br>
/Users/jessie/Dropbox/jessie/owl_run/post_processing_dada2_sequence_tables/Examine_sequence_tables.ipynb<br>
Handles inference table parsing and fasta write-out, but not summary info.<br>
<br>
<br>
4-28-21<br>
4-29-21<br>
Jessie B-T

## <font color=magenta>CHECKPOINT
##### <font color=rebeccapurple>*Import tools & load Python packages*

In [ ]:
##    BASICS

import os                #command-line like functions, for operating system interface (finding files)
import subprocess        #recommended way of running command line programs from within python
import numpy as np       #for math and arrays
import shutil            #used to move files around
import sys               #helps with reading and writing onto text files
from tqdm import tqdm    #a progress bar for for-loops (lets you see progress in actively running loops)
from time import time    #use to track time and measure how long chunks are taking to run
import shutil            #for moving files from directory to directory
import csv               #writing and reading csv

##    FOR DNA

from Bio import Seq                 #reading in and manipulating sequence data
from Bio.Seq import Seq             #function that creates a sequence object (string with associated alphabet)
                                    #Bio.Seq is a sub-part of Bio, Seq is one of the functions within Bio.Seq
from Bio import SeqIO               #reading in and saving out sequence files (such as fasta)
from Bio.SeqRecord import SeqRecord #create sequence with associated alphabet AND associated label/ID and other info
from Bio.SeqUtils import GC         #for calculating GC content

##    FOR FIGURES

import matplotlib.pyplot as plt          #basic plotting tools that let you do most of what you'll need to do, set up as a shortcut
import matplotlib                        #get ALL of matplotlib for fancier tools
                                         #add specific other matplotlib imports as needed
matplotlib.rcParams['pdf.fonttype'] = 42 #make saved vector pdf files use fonts instead of lines/shapes for text
                                         #(essential weird line for saving vector pdfs for downstream editing in illustrator)

## <font color=magenta>CHECKPOINT
##### <font color=rebeccapurple>*Define several new functions for the pipeline*

In [ ]:
#Define a function to read in file                 ### the inferences tables are saved as CSV files, tab-delimited
def csv2dict(infilename):
    """
    Description
        Read in a csv with sequencing info and build a dictionary,
        each key a column header, each value a list of strings with column
        contents (in original row order). Input file has one row for each
        specimen, with species name, identifying numbers, sequences and notes
        in columns.
    Input: File name
    Output: Data dictionary
    Example: plate1 = csv2dict(SMHF_DNA_1.csv)
    """
    #Initialize empty data dictionary
    datadict = {}
    #Open file
    print("Opening file {}".format(infilename))
    with open(infilename) as csv_file:
        """
        Uses csv DictReader tool to parse file.
        This method good at automatically dealing with excel quirks
        (line endings weird, incomplete rows not always filled out)
        """
        csv_reader = csv.DictReader(csv_file, delimiter='\t') #tabs!   since it is a tab-delimited file
        headers = csv_reader.fieldnames #What are the columns?
        """
        Default DictReader dictionary format annoying; Each row a different
        dictionary with the same headers. Make a new single dictionary, each
        column a list in a dictionary.
        """
        # Loop over headers from csv reader to make them keys of new dictionary
        for header in headers:
            datadict[header] = [] #make values empty rows for now
        # Loop over rows (dictionaries) in csv reader object, representing lines
        for row in csv_reader:
            # Loop over columns
            for header in headers:
                # Fill in lists with entry for this row,column from csv reader
                entry = row[header]
                #Remove trailing/leading whitespace (occurs in database)
                if entry is not None:
                    datadict[header].append(entry.strip())
                # Can't strip whitespace from NoneType objects of empty fields
                else:
                    datadict[header].append(entry)
    print(f"Created data dictionary. {len(datadict[headers[0]])} rows, {len(headers)} columns")
    return(datadict)

In [ ]:
#Define a function to write a data dictionary out to tab-delimited text file
def savedict(indict, outfile):
    """
    Save a dictionary of form header_key:data_list
    to tab-delimited file, given dictionary and file name string.
    Creates a table with columns representing keys and rows items in value lists.
    """
    #Get keys into a stable list
    ordered_keys = list(indict.keys())

    #Initialize list of lines to save with header line based on keys
    filerows = ["\t".join(ordered_keys)]

    #Example key (use first one)
    examplekey = ordered_keys[0]

    #Loop over rows in dataset
    for i in range(len(indict[examplekey])):
        #place to store entries for each column for this row as a list
        thisrow = []
        #Loop over columns
        for key in ordered_keys:
            thisrow.append(str(indict[key][i]))#for join method, has to be a string.

        #Join items in a file line
        filerows.append("\t".join(thisrow))

    #Save lines to file
    with open(outfile, 'w') as out:
        for line in filerows:
            out.write(line+"\n")
    # Report back
    print(f"Saving dictionary to file {outfile}, {(len(filerows)-1)} lines of data, {len(indict.keys())} columns.")
    # Return nothing
    return(None)



## <font color=magenta>CHECKPOINT
##### <font color=rebeccapurple>*Read in and format inference table data*

In [ ]:
### Read set dada2 inference table results

MICH = csv2dict(f"{UPdir}\\7d_inference_table_chimerasremoved.txt")
# also prints it out, tells how many rows and columns in the file

### Every unique sequence in the WHOLE dataset gets its own column

In [ ]:
#Look at first few keys
print(list(MICH.keys())[0:5])    # print the first keys, so the first 5 column headers
print("")
print(MICH["Filename"][0])  #print the first item in the filename column
print("")
print(list(MICH.keys())[2])                    ## for all keys in this dictionary, what is key 2 (which means the third column header here...)
print(MICH[list(MICH.keys())[2]][0:10])       ## in key 2 (column 3), what are the first 10 items (really its 0-9 which means rows 1 through 10 (after header row))
print(MICH[list(MICH.keys())[0]][0:10])       ## in key 0 (column 1), what are the first 10 items (" ")

In [ ]:
#Pull column names for sequences (the sequences!)
def getseqkeys(inference):
    sequencekeys = []
    for key in inference.keys():
        if key not in ['Specimen', 'Filename', 'UniqueSeqCount', 'TotalCount']:
            sequencekeys.append(key)
    return(sequencekeys)

MICH_seqs = getseqkeys(MICH)
print(len(MICH_seqs))

## how many unique sequences are there, so in all of the keys, how many keys are NOT the four listed above.
    # UniqueSeqCount and TotalCount do not exist quite yet, but they may in the future, so clarify.
## there are 485 columns. 483 of those columns are literal sequences, and then two are names like "Specimen" and "Filename"

In [ ]:
#Make numbers numbers in table (csv2dict leaves input as strings)     ## make sure "0" is actually literally the number zero
def str2int(inference):
    for key in getseqkeys(inference):
        newcol = []
        for item in inference[key]:
            newcol.append(int(item))
        inference[key] = newcol
    return(inference)

MICH = str2int(MICH)

In [ ]:
# Add Summary Columns:
#     How many unique sequences per specimen?
#     How many total abundance counts per specimen?

def addSums(inference):                  # define a new function called addSums
    inference['UniqueSeqCount'] = []     # making a new key (column) called UniqueSeqCount, which right now is an empty list
    inference['TotalCount'] = []         # making a new key (column) called TotalCount, which right now is an empty list
    for i in range(len(inference['Specimen'])):  # loop over the total number of rows (using the first column arbitrarily)
        this_row_nonzero_counts = []             # going to store non-zero abundance counts in this new list
        for seq in getseqkeys(inference):        # looping over the sequence columns
            if inference[seq][i]!=0:             # for each given sequence and each given specimen (one cell (columnxrow) at a time),  if the value is NOT zero ("!=0" = evaluate to true if not zero, false if zero)
                this_row_nonzero_counts.append(inference[seq][i])  # then, for those in which the value is TRUE, then add whatever that abundance count was to this list
        inference['UniqueSeqCount'].append(len(this_row_nonzero_counts)) # for this same specimen, how many items (abundance values) were found and added to that list (**how many unique sequences per specimens**)
        inference['TotalCount'].append(sum(this_row_nonzero_counts))     # for this specimen, how many sequences were there total ("how many survived pipeline")       (**how many sequences per specimen**)
    return(inference)                    # function starts with being defined, and ends here with the output being... outputted ("modified dictionary" cuz now there are two more columns)

MICH = addSums(MICH)   #MICH is being replaced by the new and modified MICH

savedict(MICH,"9_InferenceTable_SummaryCounts.txt")

In [ ]:
#Print the number of unique sequences, number of specimens, and number of specimens with no sequences for each dataset
print("Unique seqs\tSpecimens\tSpecimens w/ 0 seqs\tSpecimens w/ <10 reads\tSpecimens w/ <150 reads")
print(f"{len(getseqkeys(MICH))}\t{len(MICH['Specimen'])}\t{MICH['UniqueSeqCount'].count(0)}\t{len([i for i in MICH['TotalCount'] if i<10])}\t{len([i for i in MICH['TotalCount'] if i<150])}")

## "Specimens  w/ 0 seqs" are specimens that lost ALL sequences in processing, and didn't survive the pipeline
## "Specimens w <10 seqs" in order to look at specimens with very low counts and not a lot support it
## low read counts - less trust worthy; threshold or number is arbitrary until/unless further digging.

## <font color=magenta>CHECKPOINT
##### <font color=rebeccapurple>*Make figures of number of unique sequences per specimen!*

## <font color=magenta>CHECKPOINT
##### <font color=rebeccapurple>*Save out most abundant sequence & summary info for each specimen*
For now, getting 1 representative sequence per specimen. Take first instance (arbitrary) in cases of abundance ties. Other sequences are  relevant too, e.g. biological variants, explanations for weird stuff, but for a single sequence rep this seems like a reasonable approach. Some ways to handle this depth of other data will be to get answers for this top sequence, then follow up for problematic sequences, and do dedicated analyses for more sequences across whole dataset, but we have to start somewhere!

In [ ]:
def saveOneMostAbundantTiesFirst(inference, label):
    print(label)
    
    #Set up a dictionary to hold summary info about process & sequences
    summary = {}
    summary['Specimen'] = []                                     ## specimen name
    summary['Dataset version'] = []                              ## specify which dataset (aka OWL_22, are you doing QR, HM, what?? specify here!)
    summary['Any dada2 results?'] = []                           ## what specimens had any results at all, like who survived
    summary['Total dada2 read abundance for specimen'] = []      ## how many total reads/sequences per specimen
    summary['Total dada2 unique sequences for specimen'] = []    ## how many unique sequences per specimen (ex: 12 reads worth of one unique sequence)
    summary['Most abundant sequence'] = []                       ## which of those unique sequences had the highest abundance (most reads)
    summary['Fasta id'] = []                                     ## save the newly formed fasta file's ID to this table
    summary['Fasta file'] = []                                   ## the name of the new fasta file
    summary['Abundance of most abundant'] = []                   ## how many reads were there of that one, top unique seq
    summary['Tie for most abundant?'] = []                       ## were there multiple most-abundant? Abritrairly take the first one
    summary['Sequence length'] = []                              ## how long is this chosen sequence
    summary['Sequence GC content'] = []                          ## what is the GC content of the chosen sequence

    single_sequence_list = [] #store 1 most abundant sequence per specimen (that has any sequences) here

    #Loop over specimens for dataset
    for i in range(len(inference['Specimen'])):       #looping over the length of the table, so just loop over each row
        summary['Specimen'].append(inference['Specimen'][i]) #store specimen name (populate Specimen key)
        summary['Dataset version'].append(label)              
        summary['Total dada2 read abundance for specimen'].append(inference['TotalCount'][i])
        summary['Total dada2 unique sequences for specimen'].append(inference['UniqueSeqCount'][i])
        ## ABOVE: just copy over the values we've already found earlier in the code
        
        #Create matched lists of the sequences for this specimen and their abundances
        this_one_seqs = []         #for this specimen, what unique sequences did it come back with
        this_one_seq_counts = []   #for this specimen, what were the read counts per the above unique sequences
        for seq in getseqkeys(inference):
            thisseqthisspeccount = inference[seq][i]
            if thisseqthisspeccount != 0:    #if TRUE that it isn't zero
                this_one_seqs.append(seq)
                this_one_seq_counts.append(thisseqthisspeccount)

        #Check that there are sequences for this specimen
        if len(this_one_seqs) == 0:           # does the specimen have any sequences at all? (did it survive processing)
            #No sequences? fill up summary table with boring stuff
               ## replace zeros with NA to ignore that specimen cuz the zeros are not meaningful here
            summary['Any dada2 results?'].append('No sequences left after dada2 pipeline.')
            summary['Most abundant sequence'].append('NA')
            summary['Fasta id'].append('NA')
            summary['Fasta file'].append('NA')
            summary['Abundance of most abundant'].append('NA')
            summary['Tie for most abundant?'].append('NA')
            summary['Sequence length'].append('NA')
            summary['Sequence GC content'].append('NA')
        
        else:      #if the above IF is NOT true, then is THIS true? So, if the rwo DOES have something (and didn't get filled with NA)...
            #Find the most abundant sequence's abundance
            max_value = max(this_one_seq_counts)    # ...then find the most abundant sequence by finding the max of all the reads per unique sequence

            #Check for a tie, if exactly one instance of max:
            if this_one_seq_counts.count(max_value)==1:  # if the instance of the maximum count value is 1, then there's one. Great, not a tie.
                tienote = "Not a tie, single most abundant sequence found."
                #Get index of max value
                max_index = this_one_seq_counts.index(max_value)   # use matching index to find what the sequence is based on where in the list it is
                max_seq = this_one_seqs[max_index]                 # using the matching index, get the actual sequence itself
                #Create a fasta sequence name
                name = inference['Specimen'][i]+"_"+label+"_most_abundant_seq_"+str(max_value)+"x"  # generate a FASTA header
                #Create a sequence record
                record = SeqRecord(Seq(max_seq),id=name,name="",description="")   # turn into a sequence record (an object, not just a string) so that we can manipulate it in code later on. (description comes after the space (after the header))
                #Store sequence into the sequence list
                single_sequence_list.append(record)

            else:
                print(f"{this_one_seq_counts.count(max_value)} sequences for specimen {inference['Specimen'][i]} tied for max abundance at {max_value}x, saving first one.")
                #ABOVE: if there IS a tie somewhere, print it out. If there are no ties, nothing will be printed out here. (so if there multiple (>1) occurences of the max value)
                #Get index of max value: this gets first instance
                max_index = this_one_seq_counts.index(max_value)
                max_seq = this_one_seqs[max_index]
                #Create a fasta sequence name
                name = inference['Specimen'][i]+"_"+label+"_tied_most_abundant_seq_"+str(max_value)+"x"
                #Create a sequence record
                record = SeqRecord(Seq(max_seq),id=name,name="",description="")
                #Store sequence in a list
                single_sequence_list.append(record)
                
                
                #all sequences for max value
                all_indices = np.where(np.array(this_one_seq_counts) == max_value)[0]
                tied_seqs = []
                for z in all_indices:
                    tied_seqs.append(this_one_seqs[z])
                
                tienote = f"{this_one_seq_counts.count(max_value)} sequences tied for max abundance at {max_value}x, used first: {','.join(tied_seqs)}"

            #Populate summary table (one line per specimen) for the specimns that DID have sequences (did survive, have unique seqs)
            summary['Any dada2 results?'].append('Yes')                    # "it did survive processing"
            summary['Most abundant sequence'].append(max_seq)              # what is the sequence
            summary['Fasta id'].append(name)                               # the fasta header (variable called "name")
            summary['Abundance of most abundant'].append(max_value)        # How many reads were there of this sequence
            summary['Sequence length'].append(len(max_seq))                # length (# basepairs) of sequence
            summary['Sequence GC content'].append(f"{GC(max_seq):.1f}")    # % GC content of sequence, round to 1 decimal place
            summary['Tie for most abundant?'].append(tienote)              # provide the sentence made above if there is a tie

    #Write out sequences to fasta file
    outpath = label+f"_single_most_abundant_sequence_for_{len(single_sequence_list)}_specimens_first_instance_in_ties.fasta"   # name includes how many seqs are in it
    SeqIO.write(single_sequence_list,outpath, "fasta")   # writes out the sequences into the fasta file
    print(f"Write {len(single_sequence_list)} sequences to {outpath}")
    
    for j in range(len(inference['Specimen'])):
        summary['Fasta file'].append(outpath)              
    
    #Write out summary dictionary to tab-delimited text file
    summary_outfile = label+f"_single_most_abundant_sequence_for_{len(single_sequence_list)}_specimens_first_instance_in_ties_summary.txt"
    savedict(summary, summary_outfile)
    
    #In case you want to explore data right away, send out summary table and sequences
    return(summary, single_sequence_list)           #return summary dictionary and lists to wok with them later
    
#Run function! 
MICH_summary, MICH_most_abundant_seqs = saveOneMostAbundantTiesFirst(MICH, "MICH")

## think about how to manage everything in excel workbooks etc since there are a lot of new spreadsheets to work with here

## find % : abundant of most abundant / total abundance